## 载入包和类

In [1]:
import hashlib
import requests
import json
# RSA公钥加密
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5 as Cipher_pkcs1_v1_5
from Crypto.Signature import PKCS1_v1_5 as Signature_pkcs1_v1_5
import base64
# 基本
import numpy as np
import pandas as pd
# 时间处理
import datetime
from dateutil import rrule
from dateutil.relativedelta import relativedelta
# 绘图
import matplotlib.pyplot as plt
# 绘图支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:
class YiXiaoYuan:
    def __init__(self, phone, password, deviceId, osType):
        self.phone = phone
        self.deviceId = deviceId
        self.password = self.rsa_encrypt(self.md5(password))
        self.osType = osType
        self.login_by_password()
        self.getBaseInfo()

    # 登录密码md5加密
    def md5(self, password):
        encrypt_md5 = hashlib.md5()
        encrypt_md5.update(password.encode())
        encrypt_password = encrypt_md5.hexdigest()
        return encrypt_password

    # 登录密码RSA加密公钥
    def getPublicKey(self):
        url = 'https://compus.xiaofubao.com/login/getPublicKey'
        data = {'deviceId': self.deviceId}
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0'}
        _return = json.loads(requests.post(
            url, params=data, headers=headers).text)
        if(_return['statusCode'] != 0):
            print('[Tips]:获取PublicKey失败！')
        else:
            public_key = '''-----BEGIN PUBLIC KEY-----
'''+_return['data']['publicKey'] + '''
-----END PUBLIC KEY-----'''
            return public_key

    # 登录密码RSA加密
    def rsa_encrypt(self, password):
        public_key = self.getPublicKey()
        cipher = Cipher_pkcs1_v1_5.new(RSA.importKey(public_key))
        cipher_text = base64.b64encode(
            cipher.encrypt(password.encode())).decode()
        return cipher_text

    # 登录
    def login_by_password(self):
        url = 'https://compus.xiaofubao.com/login/doLoginByPwd'
        data = {'mobilePhone': self.phone, 'password': self.password,
                'osType': self.osType, 'deviceId': self.deviceId}
        headers = {
            'content-type': "Content-Type:application/x-www-form-urlencoded",
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0'}
        _return = json.loads(requests.post(
            url, params=data, headers=headers).text)
        if(_return['statusCode'] != 0):
            print('[Tips]:登录失败！失败原因:{}'.format(_return['message']))
        else:
            print('[Tips]:欢迎{}，登录成功！'.format(_return['data']['userName']))
            self.loginData = _return

    # 获取登录的基本信息
    def getBaseInfo(self):
        self.id = self.loginData['data']['id']
        self.token = self.loginData['data']['token']
        self.deviceId = self.loginData['data']['deviceId']
        self.schoolCode = self.loginData['data']['schoolCode']
        self.schoolName = self.loginData['data']['schoolName']
        self.userName = self.loginData['data']['userName']
        self.userType = self.loginData['data']['userType']
        self.jobNo = self.loginData['data']['jobNo']
        self.sex = self.loginData['data']['sex']
        self.userClass = self.loginData['data']['userClass']
        self.regiserTime = self.loginData['data']['regiserTime']
        self.headImg = self.loginData['data']['headImg']

    # 获取余额
    def getCardMoney(self):
        url = 'https://compus.xiaofubao.com/compus/user/getCardMoney'
        data = {'id': self.id, 'token': self.token, 'deviceId': self.deviceId}
        headers = {
            'content-type': "Content-Type:application/x-www-form-urlencoded",
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0'}
        _return = json.loads(requests.post(
            url, params=data, headers=headers).text)
        if(_return['statusCode'] != 0):
            print('[Tips]:获取失败！失败原因:{}'.format(_return['message']))
        else:
            self.CardMoney = _return['data']
            return float(self.CardMoney)

    # 获取从指定日期开始到今天的所有数据
    def queryCard(self, year, month):
        # 数据产生的最初日期，不知道就直接写个较小的
        start_time = datetime.date(year, month, 1)
        end_time = datetime.datetime.now()
        months = rrule.rrule(
            rrule.MONTHLY, dtstart=start_time, until=end_time).count()
        data_list = list()
        for i in range(months):
            year_now = str((start_time + relativedelta(months=i)).year)
            year_month = str((start_time + relativedelta(months=i)).month)
            tradeMonth = year_now+'-'+year_month
            url = 'https://compus.xiaofubao.com/routeauth/auth/route/user/cardQuerynoPage'
            data = {'tradeMonth': tradeMonth, 'id': self.id, 'token': self.token,
                    'deviceId': self.deviceId, 'platform': 'YUNMA_APP'}
            headers = {
                'content-type': "Content-Type:application/x-www-form-urlencoded",
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0'}
            _return = json.loads(requests.post(
                url, params=data, headers=headers).text)
            if(_return['total'] > 0):
                for i in _return['rows']:
                    data_list.append([i['serialno'], i['time'],
                                      i['address'], i['money']])
                print('[Tips]:{}导入成功！'.format(tradeMonth))
            else:
                print('[Tips]:{}导入失败！'.format(tradeMonth))
        df = pd.DataFrame(data_list, columns=[
                          'id', 'time', 'address', 'money'])
        df = df.sort_values('time').reset_index(drop=True)
        df["money"] = df["money"].astype("float")
        self.df = df

    # 对消费地点分组求和
    def sumbyAddress(self):
        return self.df.groupby('address', as_index=False).agg({'money': np.sum})

    def sumbyDay(self):
        # 选择支出
        df2 = df[df['money'] < 0]
        # 转换为时间，年月日
        pd.set_option('mode.chained_assignment', None)
        df2['time'] = pd.to_datetime(df2["time"], errors='coerce')
        df2['time'] = df2['time'].apply(lambda x: x.strftime("%Y-%m-%d"))
        return df2.groupby('time', as_index=False).agg({'money': np.sum})

## 登录

In [3]:
phone = ''
password = ''
deviceId = '3ED5F00A-A02C-4383-A383-4BEF38BF1604'
osType = 'ios'
obj = YiXiaoYuan(phone, password, deviceId, osType)

[Tips]:登录失败！失败原因:电话号码不能为空


AttributeError: 'YiXiaoYuan' object has no attribute 'loginData'

## 导入获取数据

In [ ]:
# 导入从2020-12月开始到今天的所有数据
obj.queryCard(2020,12)

## 所有数据

In [ ]:
df = obj.df

## 分组统计及绘图展示

### 分组地点饼图

In [ ]:
# 通过消费地点分组求和
df1 = obj.sumbyAddress()
# 只绘出支出
df1 = df1[df1['money'] < 0]
# 转为正值
df1['money'] = abs(df1['money'])
# 绘图
labels = df1['address']
sizes = df1['money']
fig, ax = plt.subplots(figsize=(14, 7))
ax.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
plt.show()

### 分组天数求和

In [ ]:
# 通过天数分组求和
df2 = obj.sumbyDay()
# 转为正值
df2['money'] = abs(df2['money'])
# 绘图
import matplotlib.ticker as ticker
fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(df2['time'],df2['money'])
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
ax.set_ylabel('消费')
plt.show()